In [1]:
# Reinitialize necessary parameters and libraries
import pandas as pd
import numpy as np
import random

# Define parameters for synthetic data
num_records = 1000
random.seed(42)

# Fixing the data creation with date-only granularity
# Generate random data with dates only for placed, shipped, and delivered dates
data = {
    "shipment_tracking_number": [f"STN{str(i).zfill(6)}" for i in range(num_records)],
    "placed_dt": np.random.choice(pd.date_range(start="2023-01-01", end="2023-12-31", freq="D"), size=num_records),
    "shipped_dt": np.random.choice(pd.date_range(start="2023-01-02", end="2023-12-31", freq="D"), size=num_records),
    "delivered_dt": np.random.choice(pd.date_range(start="2023-01-03", end="2023-12-31", freq="D"), size=num_records),
    "fc_shipped_from": np.random.choice(["FC1", "FC2", "FC3", "FC4"], size=num_records),
    "nearest_fc": np.random.choice(["FC1", "FC2", "FC3", "FC4"], size=num_records),
    "dest_zip": [str(random.randint(10000, 99999)) for _ in range(num_records)],
    "package_ct": np.random.randint(1, 5, size=num_records),
    "units": np.random.randint(1, 10, size=num_records),
}

# Convert to DataFrame
shipment_data = pd.DataFrame(data)

# Ensure shipped_dt >= placed_dt and delivered_dt >= shipped_dt
shipment_data["shipped_dt"] = shipment_data.apply(
    lambda row: max(row["shipped_dt"], row["placed_dt"]), axis=1
)
shipment_data["delivered_dt"] = shipment_data.apply(
    lambda row: max(row["delivered_dt"], row["shipped_dt"]), axis=1
)

# Calculate additional metrics for CTD and STD in days
shipment_data["CTD"] = (shipment_data["delivered_dt"] - shipment_data["placed_dt"]).dt.days
shipment_data["STD"] = (shipment_data["delivered_dt"] - shipment_data["shipped_dt"]).dt.days

# Display the first few rows of the dataset
shipment_data.head()
holiday_dataset = pd.read_csv('Holiday_Dataset.csv')

# Display the first few rows of the dataset
print(holiday_dataset.head())

# Display the data types of the dataset columns
print(holiday_dataset.dtypes)

# Convert the 'date' column in holiday_dataset to datetime and extract the date part
holiday_dataset['date'] = pd.to_datetime(holiday_dataset['date']).dt.date

# Convert the 'placed_dt' column in shipment_data to datetime and extract the date part (if not already done)
shipment_data['placed_dt'] = pd.to_datetime(shipment_data['placed_dt']).dt.date

# Merge the datasets
shipment_data = pd.merge(
    shipment_data,
    holiday_dataset,
    how="left",
    left_on="placed_dt",
    right_on="date"
)

# Fill missing holiday information with defaults (not a holiday)
shipment_data["is_holiday"].fillna(0, inplace=True)
shipment_data["is_holiday_with_operations"].fillna(0, inplace=True)

# Display the first few rows of the merged dataset
shipment_data.head()


         date  is_holiday  is_holiday_with_operations
0  2023-01-01           1                           0
1  2023-01-02           0                           0
2  2023-01-03           0                           0
3  2023-01-04           0                           0
4  2023-01-05           0                           0
date                          object
is_holiday                     int64
is_holiday_with_operations     int64
dtype: object


<ipython-input-1-0e7b314e1d97>:65: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  shipment_data["is_holiday"].fillna(0, inplace=True)
<ipython-input-1-0e7b314e1d97>:66: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

,shipment_tracking_number,placed_dt,shipped_dt,delivered_dt,fc_shipped_from,nearest_fc,dest_zip,package_ct,units,CTD,STD,date,is_holiday,is_holiday_with_operations
0,STN000000,2023-02-07,2023-10-21,2023-10-21,FC3,FC4,93810,1,8,256,0,2023-02-07,0,0
1,STN000001,2023-05-22,2023-05-22,2023-05-22,FC4,FC1,24592,4,2,0,0,2023-05-22,0,0
2,STN000002,2023-12-02,2023-12-02,2023-12-09,FC4,FC2,13278,2,8,7,7,2023-12-02,0,0
3,STN000003,2023-08-13,2023-08-23,2023-08-23,FC2,FC1,46048,2,1,10,0,2023-08-13,0,0
4,STN000004,2023-04-08,2023-10-05,2023-11-19,FC4,FC2,42098,4,5,225,45,2023-04-08,0,0


In [2]:
!pip install jupyter-dash
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import numpy as np

# Create the Dash app
app = JupyterDash(__name__)

# Layout of the app
app.layout = html.Div([
    dcc.Tabs([
        # Tab 1: Filter by dest_zip
        dcc.Tab(label="Filter by Zip", children=[
            html.Div([
                html.Label("Select Zip Code(s):"),
                dcc.Dropdown(
                    id="zip_filter",
                    options=[{"label": zip_code, "value": zip_code} for zip_code in shipment_data["dest_zip"].unique()],
                    multi=True,
                    placeholder="Select one or more Zip Codes",
                ),
                html.Label("Select Date Range:"),
                dcc.DatePickerRange(
                    id="date_filter_zip",
                    start_date=shipment_data["placed_dt"].min(),
                    end_date=shipment_data["placed_dt"].max(),
                ),
                dcc.Graph(id="line_chart_zip"),
            ])
        ]),
        # Tab 2: Filter by fc_shipped_from
        dcc.Tab(label="Filter by Fulfillment Center", children=[
            html.Div([
                html.Label("Select Fulfillment Center(s):"),
                dcc.Dropdown(
                    id="fc_filter",
                    options=[{"label": fc, "value": fc} for fc in shipment_data["fc_shipped_from"].unique()],
                    multi=True,
                    placeholder="Select one or more Fulfillment Centers",
                ),
                html.Label("Select Date Range:"),
                dcc.DatePickerRange(
                    id="date_filter_fc",
                    start_date=shipment_data["placed_dt"].min(),
                    end_date=shipment_data["placed_dt"].max(),
                ),
                dcc.Graph(id="line_chart_fc"),
            ])
        ]),
        # Tab 3: Boxplots for CTD and STD by Day of Week
        dcc.Tab(label="CTD and STD Boxplots", children=[
            html.Div([
                html.Label("Average CTD and STD by Day of the Week"),
                dcc.Graph(
                    figure=px.box(
                        shipment_data.assign(day_of_week=pd.to_datetime(shipment_data["placed_dt"]).dt.day_name())
                        .melt(id_vars=["day_of_week"], value_vars=["CTD", "STD"], var_name="Metric", value_name="Value"),
                        x="day_of_week", y="Value", color="Metric",
                        title="CTD and STD Comparison by Day of the Week",
                        labels={"day_of_week": "Day of Week", "Value": "Days", "Metric": "Metric"},
                        category_orders={"day_of_week": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]}
                    ).update_layout(
                        height=600,
                        xaxis_title="Day of the Week",
                        yaxis_title="Days",
                        showlegend=True,
                        boxmode="group"  # Ensures CTD and STD are grouped side-by-side for comparison
                    )
                )
            ])
        ]),
    ])
])

# Callback for line chart on Tab 1 (Filter by Zip)
@app.callback(
    Output("line_chart_zip", "figure"),
    [Input("zip_filter", "value"),
     Input("date_filter_zip", "start_date"),
     Input("date_filter_zip", "end_date")]
)
def update_chart_zip(selected_zips, start_date, end_date):
    filtered_data = shipment_data
    if selected_zips:
        filtered_data = filtered_data[filtered_data["dest_zip"].isin(selected_zips)]
    if start_date and end_date:
        filtered_data = filtered_data[
            (filtered_data["placed_dt"] >= pd.to_datetime(start_date).date()) &
            (filtered_data["placed_dt"] <= pd.to_datetime(end_date).date())
        ]
    avg_metrics = filtered_data.groupby("placed_dt").agg({"CTD": "mean", "STD": "mean"}).reset_index()

    fig = px.line(avg_metrics, x="placed_dt", y=["CTD", "STD"],
                  labels={"value": "Hours", "variable": "Metric"},
                  title="Average CTD and STD by Date (Filtered by Zip)")

    # Add vertical lines for holidays
    for _, row in filtered_data[filtered_data["is_holiday"] == 1].iterrows():
        color = "black" if row["is_holiday_with_operations"] == 1 else "green"
        fig.add_shape(
            type="line",
            x0=row["placed_dt"], x1=row["placed_dt"],
            y0=0, y1=1,
            xref="x", yref="paper",
            line=dict(color=color, width=2)
        )
    return fig

# Callback for line chart on Tab 2 (Filter by Fulfillment Center)
@app.callback(
    Output("line_chart_fc", "figure"),
    [Input("fc_filter", "value"),
     Input("date_filter_fc", "start_date"),
     Input("date_filter_fc", "end_date")]
)
def update_chart_fc(selected_fcs, start_date, end_date):
    filtered_data = shipment_data
    if selected_fcs:
        filtered_data = filtered_data[filtered_data["fc_shipped_from"].isin(selected_fcs)]
    if start_date and end_date:
        filtered_data = filtered_data[
            (filtered_data["placed_dt"] >= pd.to_datetime(start_date).date()) &
            (filtered_data["placed_dt"] <= pd.to_datetime(end_date).date())
        ]
    avg_metrics = filtered_data.groupby("placed_dt").agg({"CTD": "mean", "STD": "mean"}).reset_index()

    fig = px.line(avg_metrics, x="placed_dt", y=["CTD", "STD"],
                  labels={"value": "Hours", "variable": "Metric"},
                  title="Average CTD and STD by Date (Filtered by Fulfillment Center)")

    # Add vertical lines for holidays
    for _, row in filtered_data[filtered_data["is_holiday"] == 1].iterrows():
        color = "black" if row["is_holiday_with_operations"] == 1 else "green"
        fig.add_shape(
            type="line",
            x0=row["placed_dt"], x1=row["placed_dt"],
            y0=0, y1=1,
            xref="x", yref="paper",
            line=dict(color=color, width=2)
        )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8050)


/usr/local/lib/python3.10/dist-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

Dash app running on:
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>